***GENERATED CODE FOR heartfailurepredictionmodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    Decision_tree_classifier = DecisionTreeClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    Decision_tree_classifier.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            Decision_tree_classifier.score(X_test, y_test))

    data = {'model': Decision_tree_classifier,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run heartfailurepredictionmodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	heartfailureclinicalrecorddataet = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1639660053458_heart_failure_clinical_records_dataset.csv', 'filename': 'heart_failure_clinical_records_dataset.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Healthcare Models/Heart Failure - Classification/heart_failure_clinical_records_dataset.csv', 'viewFileName': 'heart_failure_clinical_records_dataset.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.139.69.95:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run heartfailurepredictionmodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(heartfailureclinicalrecorddataet,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "15", "mean": "60.13", "stddev": "12.89", "min": "40.0", "max": "80.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "anaemia", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "0.27", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "anaemia"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine_phosphokinase", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "1017.73", "stddev": "1311.85", "min": "90", "max": "5209", "missing": "0"}, "updatedLabel": "creatinine_phosphokinase"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "diabetes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "0.27", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "diabetes"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ejection_fraction", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "33.53", "stddev": "11.03", "min": "15", "max": "60", "missing": "0"}, "updatedLabel": "ejection_fraction"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "high_blood_pressure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "0.4", "stddev": "0.51", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "high_blood_pressure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "platelets", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "15", "mean": "266157.2", "stddev": "106771.99", "min": "127000.0", "max": "504000.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "platelets"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "serum_creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "15", "mean": "1.01", "stddev": "0.21", "min": "0.6", "max": "1.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "serum_creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "serum_sodium", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "137.07", "stddev": "4.18", "min": "125", "max": "144", "missing": "0"}, "updatedLabel": "serum_sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sex", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "0.67", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "sex"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "smoking", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "0.4", "stddev": "0.51", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "smoking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "time", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "136.0", "stddev": "72.99", "min": "10", "max": "245", "missing": "0"}, "updatedLabel": "time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DEATH_EVENT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "15", "mean": "0.2", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "DEATH_EVENT"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run heartfailurepredictionmodelHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=DecisionTreeClassifier(manualfe, ["age", "anaemia", "creatinine_phosphokinase", "diabetes", "ejection_fraction", "high_blood_pressure", "platelets", "serum_creatinine", "serum_sodium", "sex", "smoking", "time"], "DEATH_EVENT")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
